In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=0a53cbf27608a3c46fbfd39f46304616fb77110c4f051006774750d1a5663c98
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import json
from google.colab import drive
import pandas as pd

In [ ]:
drive.mount('/content/drive')

post_path="/content/drive/MyDrive/TFM/Embeddings/Posts.json"

d = pd.read_csv('/content/drive/MyDrive/TFM/Embeddings/SentenceEmbeddings.csv', header=None)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
done = set(d.iloc[:, 0].tolist())


In [ ]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-small')

with open(post_path, "r") as input:
  #inp = json.load(input)
  ids = []
  caption_batch = []
  i =1
  for line in input:
    line = json.loads(line)
    if line["id"] not in done:
      ids.append(line["id"])
      caption_batch.append(f'query: {line["caption"]}')
      if i %5 == 0:
        try:
          batch_dict = tokenizer(caption_batch, max_length=512, padding=True, truncation=True, return_tensors='pt')
          outputs = model(**batch_dict)
          a = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

          embeddings = a.detach().numpy()
          df_features = pd.DataFrame(embeddings)
          df_ids = pd.DataFrame(ids)
          merged_df = pd.concat((df_ids, df_features), axis=1)
          merged_df.to_csv('/content/drive/MyDrive/TFM/Embeddings/SentenceEmbeddings.csv',  mode='a', header=False, index=False)

          caption_batch = []
          ids = []
        except:
          print(caption_batch)

      i +=1

  batch_dict = tokenizer(caption_batch, max_length=512, padding=True, truncation=True, return_tensors='pt')
  outputs = model(**batch_dict)
  a = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
  embeddings = a.detach().numpy()
  df_features = pd.DataFrame(embeddings)
  df_ids = pd.DataFrame(ids)
  merged_df = pd.concat((df_ids, df_features), axis=1)
  merged_df.to_csv('/content/drive/MyDrive/TFM/Embeddings/SentenceEmbeddings.csv',  mode='a', header=False, index=False)


IndexError: ignored

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFM/Embeddings/SentenceEmbeddings.csv', header=None)

# Drop duplicates based on all columns
df.drop_duplicates(subset=df.columns[0], inplace=True)

df.to_csv('/content/drive/MyDrive/TFM/Embeddings/SentenceEmbeddings.csv',  mode='w', header=False, index=False)